# Parametrisation of gemmae contours


## Documentation and Imports


Created on 27-05-2021

Author: Valentin Laplaud

This code loads gemmae contours from ShapeComputation.ipynb and express them as a function of a curvilinar abcisse. Adding landmark points (notches and attach points) then allows a deformation of the contour to match the position of landmark points.

In [ ]:
## Clean up before script start 

for element in dir():
    if element[0:2] != "__":
        del globals()[element]
import gc
gc.collect()



print('\033[1m' + '\033[4m' + '\nRunning ''ShapeParametrisationNB'' \n' + '\033[0m')

# plotting stuff
import matplotlib as mpl
%matplotlib inline
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2
mpl.rcParams['axes.facecolor'] = COLOR2

from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
from cycler import cycler
#Default colors
colorcycle = [plt.get_cmap('gist_rainbow')(1. * i/30) for i in range(30)]
mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)

# numbers handling
import numpy as np
import numpy.matlib as mtl
import pandas as pd
    

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags, find_peaks
from scipy.interpolate import interp1d, UnivariateSpline
from scipy import optimize

# images handling
from skimage import io
from skimage.filters import threshold_otsu, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv

# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../')
import VallapFunc as vf



##  Define analysis functions

### Getting landmark points 
User input is used to identify three landmarks points on the first immage (attach point and two notches). Then a curvature computation is used on every subsequent image to get points of high inwards curvature. The position of the three landmarks in time is then propagated from the first image to the nearest point of high curvature.

#### Identifiying contour points from user input.

In [ ]:
def getContourPointsCoordinates(P,s,npts,nimg,X,Y,Xc,Yc,Xold,Yold,Xcold,Ycold):            
        
    RGBimg = io.imread(P + '\\' + s + '.tif', key = nimg) # get the  image from tiff stack

    # ask user to click
    %matplotlib qt
    plt.figure(dpi=250)
    plt.imshow(RGBimg)
    plt.plot(X,Y,'c',lw = 0.7)
    plt.plot(Xold,Yold,'b--',lw=0.3)
    plt.plot(Xcold,Ycold,'mo',ms = 2)
    plt.plot(Xc,Yc,'r+',ms = 1.5)
    pts = np.asarray(plt.ginput(npts, timeout=-1))
    plt.close()
    %matplotlib inline

    Points = pts[:]

    for i in range(npts):
        x = pts[i][0]
        y = pts[i][1]

        xc = Xc[np.argmin(np.sqrt(np.square(Xc-x)+np.square(Yc-y)))]            
        yc = Yc[np.argmin(np.sqrt(np.square(Xc-x)+np.square(Yc-y)))]

        Points[i][0] = xc
        Points[i][1] = yc

    return(Points)
 

#### Computing contour curvature unsing circle fitting

In [ ]:
def fitCircle(X,Y):
    
    x_m = np.mean(X)
    y_m = np.mean(Y)    
    
    def calc_R(xc, yc):

        return np.sqrt((X - xc) ** 2 + (Y - yc) ** 2)
    
    def f_2(c):
        
        Ri = calc_R(*c)
        return Ri - Ri.mean()
    
    center_estimate = x_m, y_m
    center_fit, _ = optimize.leastsq(f_2, center_estimate)

    xc_fit, yc_fit = center_fit
    Ri_fit       = calc_R(xc_fit, yc_fit)
    
     #Fitting the radius of the circle
    R_fit        = Ri_fit.mean()
   
    
    if False:
        fig,ax = plt.subplots(dpi=250, facecolor = 'black')
        ax.plot(X,Y,'ro')

        xcircle = xc_fit + R_fit*cos(np.linspace(-pi,pi,100))
        ycircle = yc_fit + R_fit*sin(np.linspace(-pi,pi,100))
        ax.plot(xcircle,ycircle,'k--')

    return([xc_fit, yc_fit,R_fit])
    
def getContourCurvature(X,Y,step):

    newX = np.concatenate((X[-step:],X,X[:step]))    
    newY = np.concatenate((Y[-step:],Y,Y[:step]))
    
    Curv = []
    xc = []
    yc = []
    
    for i in range(len(X)):
        a = fitCircle(newX[i:i+2*step],newY[i:i+2*step])
        xc.append(a[0])
        yc.append(a[1])
        Curv.append(1/a[2])
        
    return(xc,yc,Curv)


#### Smoothing contour and getting landmarks 

In [ ]:
def getLandmarks(CD,GD,StackList,P, **kwargs): 
        
    # Utility function
    def round_to_odd(f):
        return np.floor(f) // 2 * 2 + 1    
    
    # Folder definition
    if not os.path.exists(P + '\\Figures\\'):
        os.mkdir(P + '\\Figures\\') # create folder
        
    if not os.path.exists(P + '\\Figures\\Landmarks\\'):
        os.mkdir(P + '\\Figures\\Landmarks\\') # create folder
        
    # Kwargs
    DebugPlots = False
    SavedPlots = False
    Dmax = 20
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'saveplots':
            SavedPlots = value
        elif key == 'Dmax':
            Dmax = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
        
    # Load or create file for landmark points 
    if os.path.exists(P + '\\clickedpoints.csv'):
        clickyshitsaving = pd.read_csv(P + '\\clickedpoints.csv', index_col = 'Ind')
    else:                
        clickyshitsaving = pd.DataFrame(data=None,columns=['Img','Xnotch1','Ynotch1','Xnotch2','Ynotch2','Xattach','Yattach']) 
    
    # 1. For each video in StackList identify landmarks on first image
    
    for s in StackList:
        
        if not os.path.exists(P + '\\Figures\\Landmarks\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Landmarks\\'+ s + '\\') # create folder
            
        print('First image landmarks for : ' + s.ljust(10), flush=True, end = '\r')
        
        i = 0 # First image
        
        # Loading contour and center coordinate
        Xcenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xcenter'].values
        Ycenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ycenter'].values

        Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ximg'].values
        Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Yimg'].values

        # Contour smoothing
        window = int(round_to_odd(len(Xcont)/50))

        Xsmooth = savgol_filter(Xcont, window, 3)
        Ysmooth = savgol_filter(Ycont, window, 3)

        # Save smoothed contour
        CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Xsmooth'] = Xsmooth
        CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ysmooth'] = Ysmooth
        
        ### Automaticaly detect points of high inward curvature  
        # Compute curvature
        Step = round(2/100*len(Xsmooth))
        xc,yc,Curv = getContourCurvature(Xsmooth,Ysmooth,Step)

        # Compute signed curvature using the position of the center of the fitted 
        # circle with regards to the contour
        poly = [(x,y) for (x,y) in zip(Xsmooth,Ysmooth)]
        points = [[x,y] for (x,y) in zip(xc,yc)]
        path = mpltPath.Path(poly)            
        isIn = path.contains_points(points)

        CurvSign = []
        for Bool in isIn:
            if Bool:
                CurvSign.append(1)
            else:
                CurvSign.append(-1)

        SignedCurv = np.multiply(Curv,CurvSign)

        # Detecting negative peaks in signed curvature
        loc,prop = find_peaks(-SignedCurv, distance = len(SignedCurv)/30) 


        ### Identifying landmarks
        # if clicked points not already in the file, ask user and then save in the file
        if clickyshitsaving.loc[(clickyshitsaving.index == s)&(clickyshitsaving['Img']==i)].empty:

            # identifying notches manually on fist image
            NotchesRef = getContourPointsCoordinates(P,s,2,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc])
            AttachRef = getContourPointsCoordinates(P,s,1,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc])

            # Save the points that were clicked
            data = {'Img': i,
                    'Xnotch1':NotchesRef[0][0],
                    'Ynotch1':NotchesRef[0][1],
                    'Xnotch2':NotchesRef[1][0],
                    'Ynotch2':NotchesRef[1][1],
                    'Xattach':AttachRef[0][0],
                    'Yattach':AttachRef[0][1]}            

            clickyshitsaving = clickyshitsaving.append(pd.DataFrame(data=data,index = [s]))

            clickyshitsaving.to_csv(P + '\\clickedpoints.csv',index_label = 'Ind')
        
        else: # Load clicked points from the file
                NotchesRef = [[clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xnotch1'].values,
                                   clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Ynotch1'].values],
                                 [clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xnotch2'].values,
                                   clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Ynotch2'].values]]
                AttachRef = [[clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xattach'].values, 
                             clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Yattach'].values]]
                    
        
        # Find landmarks in contour from clicked points and Curvature computation   
        xN1 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                              np.square(Ysmooth[loc]-NotchesRef[0][1])))]  
        yN1 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                              np.square(Ysmooth[loc]-NotchesRef[0][1])))] 

        xN2 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                              np.square(Ysmooth[loc]-NotchesRef[1][1])))]  
        yN2 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                              np.square(Ysmooth[loc]-NotchesRef[1][1])))] 

        xA = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                              np.square(Ysmooth[loc]-AttachRef[0][1])))]  
        yA = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                              np.square(Ysmooth[loc]-AttachRef[0][1])))] 

        # Save landmarks points
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xattach'] = xA
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Yattach'] = yA

        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xnotch1'] = xN1
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ynotch1'] = yN1

        GD.loc[(GD.index == s) & (GD['Img'] == i),'Xnotch2'] = xN2
        GD.loc[(GD.index == s) & (GD['Img'] == i),'Ynotch2'] = yN2

        # create and save initial parametrisation
        npts = len(Xsmooth)
        S = np.linspace(0,1,npts)

        CD.loc[(CD.index == s) & (CD['Img'] == i) ,'InitialS'] = S

        # Find and save the positions in contour and parametric indexes of notches points
        locn1 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN1)+np.square(Ysmooth-yN1))))
        locn2 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN2)+np.square(Ysmooth-yN2))))

        Sn1 = S[locn1]
        Sn2 = S[locn2]

        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1'] = locn1
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2'] = locn2
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch1'] = Sn1
        GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2'] = Sn2

        # Plotting 
        if SavedPlots|DebugPlots:
            RGBimg = io.imread(P + '\\' + s + '.tif', key = i)

            fig0, [ax0, ax1] = plt.subplots(ncols = 2, dpi = 250,facecolor='black')
            fig0.suptitle(s)
            ax0.set_title('Attach point (magenta), notches (red),\n and center (blue) for alignement',fontsize=8)
            ax0.imshow(RGBimg)
            ax0.plot(Xsmooth,Ysmooth,'c',lw = 0.5)
            ax0.plot(xN1,yN1,'.r',ms=2)
            ax0.plot(xN2,yN2,'.r',ms=2)
            ax0.plot(xA,yA,'.m',ms=2)
            ax0.plot(Xcenter[0],Ycenter[0],'*b', ms = 3)
            ax0.set_xticks([], [])
            ax0.set_yticks([], [])
            sc = ax1.scatter(Xsmooth,-Ysmooth+2*Ycenter[0], c= SignedCurv, cmap = 'BrBG', s = 2)
            ax1.plot(xN1,-yN1+2*Ycenter[0],'ob',ms=3)
            ax1.plot(xN2,-yN2+2*Ycenter[0],'ob',ms=3)
            ax1.plot(xA,-yA+2*Ycenter[0],'sb',ms=3)
            ax1.plot(Xsmooth[loc],-Ysmooth[loc]+2*Ycenter[0],'r+',mfc='none', ms = 3, lw= 0.5)
            ax1.set_aspect('equal')
            ax1.set_xticks([], [])
            ax1.set_yticks([], [])
            #ax1.set_xlim(left=-250+Xcenter[0],right=250+Xcenter[0])
            #ax1.set_ylim(top=250+Ycenter[0],bottom=-250+Ycenter[0])
            fig0.colorbar(sc, ax = ax1, label = 'Curvature',shrink = 0.6)
            fig0.tight_layout()

            fig0.savefig(P + '\\Figures\\Landmarks\\'+ s + '\\' + str(i) +'.png') 
            if DebugPlots & (i == 0):
                plt.show()
            else:
                plt.close()
        
    # 2. For the remaining image, detect automatically landmark points
    for s in StackList:       
            
        nimg = int(1 + np.max(GD.loc[s, 'Img'])) # number of images in the stack
        
        for i in range(1,nimg):
            
            print('Processing ' + s + ' image ' + str(i+1) + '/' + str(nimg).ljust(10), flush=True, end = '\r')
            
            # Loading contour and center coordinate
            Xcenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xcenter'].values
            Ycenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ycenter'].values

            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ximg'].values
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Yimg'].values

            # Contour smoothing
            window = int(round_to_odd(len(Xcont)/50))

            Xsmooth = savgol_filter(Xcont, window, 3)
            Ysmooth = savgol_filter(Ycont, window, 3)

            # Save smoothed contour
            CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Xsmooth'] = Xsmooth
            CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ysmooth'] = Ysmooth
            
            # Center smoothed contour on (0,0)
            Xcent = Xsmooth - Xcenter
            Ycent = Ysmooth - Ycenter
            
            ### Automaticaly detect points of high inward curvature  
            # Compute curvature
            Step = round(2/100*len(Xsmooth))
            xc,yc,Curv = getContourCurvature(Xsmooth,Ysmooth,Step)
            
            # Compute signed curvature using the position of the center of the fitted 
            # circle with regards to the contour
            poly = [(x,y) for (x,y) in zip(Xsmooth,Ysmooth)]
            points = [[x,y] for (x,y) in zip(xc,yc)]
            path = mpltPath.Path(poly)            
            isIn = path.contains_points(points)
            
            CurvSign = []
            for Bool in isIn:
                if Bool:
                    CurvSign.append(1)
                else:
                    CurvSign.append(-1)
            
            SignedCurv = np.multiply(Curv,CurvSign)
            
            # Detecting peaks in signed curvature
            loc,prop = find_peaks(-SignedCurv, distance = len(SignedCurv)/30)             

            ### Identifying landmarks
            # if there is no reference point in the file
            if clickyshitsaving.loc[(clickyshitsaving.index == s)&(clickyshitsaving['Img']==i)].empty:
                
                # Getting centered contour from previous image 
                XsmoothOld = CD.loc[(CD.index == s) & (CD['Img'] == i-1) ,'Xsmooth'].values
                YsmoothOld = CD.loc[(CD.index == s) & (CD['Img'] == i-1) ,'Ysmooth'].values
                
                XcenterOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xcenter'].values
                YcenterOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ycenter'].values
                
                XcentOld = XsmoothOld-XcenterOld
                YcentOld = YsmoothOld-YcenterOld
                
                ### Rotate new contour to align on old one
                # Expressed in polar coordinates
                Theta,R = vf.ToCirc(XcentOld,YcentOld,angle = 'deg')
                Rs = mtl.repmat(R,360,1)
                                 
                # Rotation steps (1°)
                rot = np.transpose(np.array([range(0,360)])/2)
                Thetas = Theta + rot
                
                Xmatrot, Ymatrot = vf.ToCart(Thetas,Rs, angle = 'deg')
                
                Hvect = np.empty(360)
                
                for d in range(360):
                    Hvect[d] = vf.HausdorffDist(Xmatrot[d,:],Ymatrot[d,:],Xcent,Ycent)
                
                m = np.argmin(Hvect)
                
                XalOld = Xmatrot[m,:]+Xcenter
                YalOld = Ymatrot[m,:]+Ycenter
                
                if DebugPlots:
                    fig,ax = plt.subplots(dpi=200)
                    ax.set_title('Contour alignement for image : ' +str(i) )
                    ax.plot(XsmoothOld,YsmoothOld,'b--',label = ('i-1 contour'))
                    ax.plot(Xsmooth,Ysmooth,'w--', label = ('i contour'))
                    ax.plot(XalOld,YalOld,'g',label = ('translated and rotated i-1 contour'))
                    plt.legend()

                
                
                ### if high curvature points are too far from old 
                ### aligned contour landmarks, ask again the user
                
                # landmark points on old contour, centered
                xN1Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch1'].values-XcenterOld
                yN1Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch1'].values-YcenterOld
                xN2Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch2'].values-XcenterOld
                yN2Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch2'].values-YcenterOld
                xAOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xattach'].values-XcenterOld
                yAOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Yattach'].values-YcenterOld


                # rotation of old landmark points
                ThetaN1Old,RN1Old = vf.ToCirc(xN1Old,yN1Old, angle = 'deg')
                xN1OldRot,yN1OldRot = vf.ToCart(ThetaN1Old+m,RN1Old, angle = 'deg')
                ThetaN2Old,RN2Old = vf.ToCirc(xN2Old,yN2Old, angle = 'deg')
                xN2OldRot,yN2OldRot = vf.ToCart(ThetaN2Old+m,RN2Old, angle = 'deg')
                ThetaAOld,RAOld = vf.ToCirc(xAOld,yAOld, angle = 'deg')
                xAOldRot,yAOldRot = vf.ToCart(ThetaAOld+m,RAOld, angle = 'deg')
                
                # Old landmark points aligned to new contour
                xN1OldAl = xN1OldRot+Xcenter
                yN1OldAl = yN1OldRot+Ycenter
                xN2OldAl = xN2OldRot+Xcenter
                yN2OldAl = yN2OldRot+Ycenter
                xAOldAl = xAOldRot+Xcenter
                yAOldAl = yAOldRot+Ycenter
                

                DN1 = np.min(vf.dist(xN1OldAl,yN1OldAl,Xsmooth[loc],Ysmooth[loc]))
                DN2 = np.min(vf.dist(xN2OldAl,yN2OldAl,Xsmooth[loc],Ysmooth[loc]))
                DN = np.min([DN1,DN2])
                
                DA = np.min(vf.dist(xAOldAl,yAOldAl,Xsmooth[loc],Ysmooth[loc]))
                
                if DN > Dmax:

                    OldXns = [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch1'].values,
                             [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch2'].values]]
                    OldYns = [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch1'].values,
                             [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch2'].values]]
                    
                    NotchesRef = getContourPointsCoordinates(P,s,2,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],XsmoothOld,YsmoothOld,OldXns,OldYns)
                    
                else:
                    # aligned old landlark points (= new reference points)
                    NotchesRef = [[xN1OldAl,yN1OldAl],
                                 [xN2OldAl,yN2OldRAl]]
                
                if DA > Dmax:

                    OldXa  =  [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xattach'].values]
                    OldYa  = [GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Yattach'].values]

                    AttachRef = getContourPointsCoordinates(P,s,1,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],XsmoothOld,YsmoothOld,OldXa,OldYa)

                else:                    
                    # aligned old landlark points (= new reference points)
                    AttachRef = [[xAOldAl,yAOldAl]]
                    
                    

                # Save the reference points
                data = {'Img': i,
                        'Xnotch1':NotchesRef[0][0],
                        'Ynotch1':NotchesRef[0][1],
                        'Xnotch2':NotchesRef[1][0],
                        'Ynotch2':NotchesRef[1][1],
                        'Xattach':AttachRef[0][0],
                        'Yattach':AttachRef[0][1]}            
            
                clickyshitsaving = clickyshitsaving.append(pd.DataFrame(data=data,index = [s]))
            
                clickyshitsaving.to_csv(P + '\\clickedpoints.csv',index_label = 'Ind')
                
            else:
                NotchesRef = [[clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xnotch1'].values,
                                   clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Ynotch1'].values],
                                 [clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xnotch2'].values,
                                   clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Ynotch2'].values]]
                AttachRef = [[clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Xattach'].values, 
                             clickyshitsaving.loc[(clickyshitsaving.index == s) & (clickyshitsaving['Img'] == i) ,'Yattach'].values]]
                    

            # Saving landmarks    
            xN1 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[0][1])))]  
            yN1 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[0][1])))] 
            
            xN2 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[1][1])))]  
            yN2 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[1][1])))] 
            
            xA = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                  np.square(Ysmooth[loc]-AttachRef[0][1])))]  
            yA = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                  np.square(Ysmooth[loc]-AttachRef[0][1])))] 
            
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xattach'] = xA
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Yattach'] = yA

            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xnotch1'] = xN1
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ynotch1'] = yN1

            GD.loc[(GD.index == s) & (GD['Img'] == i),'Xnotch2'] = xN2
            GD.loc[(GD.index == s) & (GD['Img'] == i),'Ynotch2'] = yN2

            # create and save initial parametrisation
            npts = len(Xsmooth)            
            S = np.linspace(0,1,npts)

            CD.loc[(CD.index == s) & (CD['Img'] == i) ,'InitialS'] = S
            
            # Find and save position and parametric index of notches 
            locn1 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN2)+np.square(Ysmooth-yN2))))
            locn2 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN1)+np.square(Ysmooth-yN1))))
            
            Sn1 = S[locn1]
            Sn2 = S[locn2]
            
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1'] = locn1
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2'] = locn2
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch1'] = Sn1
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2'] = Sn2
            
            # plotting
            if SavedPlots|DebugPlots:
                RGBimg = io.imread(P + '\\' + s + '.tif', key = i)
                
                fig0, [ax0, ax1] = plt.subplots(ncols = 2, dpi = 250,facecolor='black')
                fig0.suptitle(s)
                ax0.set_title('Attach point (magenta), notches (red),\n and center (blue) for alignement',fontsize=8)
                ax0.imshow(RGBimg)
                ax0.plot(Xsmooth,Ysmooth,'c',lw = 0.5)
                ax0.plot(xN1,yN1,'.r',ms=2)
                ax0.plot(xN2,yN2,'.r',ms=2)
                ax0.plot(xA,yA,'.m',ms=2)
                ax0.plot(Xcenter[0],Ycenter[0],'*b', ms = 3)
                ax0.set_xticks([], [])
                ax0.set_yticks([], [])
                sc = ax1.scatter(Xsmooth,-Ysmooth+2*Ycenter[0], c= SignedCurv, cmap = 'BrBG', s = 2)
                ax1.plot(xN1,-yN1+2*Ycenter[0],'ob',ms=3)
                ax1.plot(xN2,-yN2+2*Ycenter[0],'ob',ms=3)
                ax1.plot(xA,-yA+2*Ycenter[0],'sb',ms=3)
                ax1.plot(Xsmooth[loc],-Ysmooth[loc]+2*Ycenter[0],'r+',mfc='none', ms = 3, lw= 0.5)
                ax1.set_aspect('equal')
                ax1.set_xticks([], [])
                ax1.set_yticks([], [])
                #ax1.set_xlim(left=-250+Xcenter[0],right=250+Xcenter[0])
                #ax1.set_ylim(top=250+Ycenter[0],bottom=-250+Ycenter[0])
                fig0.colorbar(sc, ax = ax1, label = 'Curvature',shrink = 0.6)
                fig0.tight_layout()

                fig0.savefig(P + '\\Figures\\Landmarks\\'+ s + '\\' + str(i) +'.png') 
                if DebugPlots & (i == 0):
                    plt.show()
                else:
                    plt.close()
                
        print('\n')

    return(CD,GD)

### Reorienting contours and landmarks

Reorientation is done by first centering the contour on the origin, then rotating until alignement of the bissector of the notches-center axes with the horizontal.

In [ ]:
def rotateAndCenterShape(CD,GD,StackList,P,Scale, **kwargs):
    
    if not os.path.exists(P + '\\Figures\\Rotation\\'):
        os.mkdir(P + '\\Figures\\Rotation\\') # create folder
    
    DebugPlots = False
    SavedPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'saveplots':
            SavedPlots = value        
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    newCD = pd.DataFrame(data=None,columns=['Img','Ximg','Yimg','XAligned','YAligned','Xsmooth','Ysmooth','InitialS']) 
    
    for s in StackList:
        
        
        if not os.path.exists(P + '\\Figures\\Rotation\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Rotation\\'+ s + '\\') # create folder
        
        n = int(1 + np.max(CD.loc[s, 'Img']))
        
        for i in range(n):
            # retrieve important points and contours
            Xc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xcenter'].values
            Yc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ycenter'].values

            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Xsmooth'].values-Xc
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Ysmooth'].values-Yc

            XA = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xattach'].values-Xc
            YA = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Yattach'].values-Yc

            Xn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1'].values-Xc
            Yn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1'].values-Yc

            Xn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2'].values-Xc
            Yn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2'].values-Yc


            A,R = vf.ToCirc([XA,Xn1,Xn2],[YA,Yn1,Yn2],angle='deg')
            Theta,Radius = vf.ToCirc(Xcont,Ycont,angle='deg')
            
            Anotches = np.mod(A[1:3].T-np.array([0,180]),360)
            
            Abis = np.mod(Anotches.sum()/2,360)-180
            
            [XAal,Xn1al,Xn2al],[YAal,Yn1al,Yn2al] = vf.ToCart(A-Abis,R,angle='deg')
            
            if YAal > 0:
                Abis = Abis + 180
                [XAal,Xn1al,Xn2al],[YAal,Yn1al,Yn2al] = vf.ToCart(A-Abis,R,angle='deg')

            Xcontal,Ycontal = vf.ToCart(Theta-Abis,Radius,angle='deg')

            # Start abcisse from attach points
            locA = np.argmin(np.abs(np.sqrt(np.square(Xcontal-XAal)+np.square(Ycontal-YAal))))
            
            Xcontal = np.roll(Xcontal,-locA)
            Ycontal = np.roll(Ycontal,-locA)

            # Storing contour data
            data = {'Img': i*np.ones(len(Xcontal)), 
                        'Ximg': Xcont+Xc,  
                        'Yimg': Ycont+Yc,
                        'Xsmooth': CD.loc[(CD.index == s) & (CD['Img'] == i), 'Xsmooth'].values,  
                        'Ysmooth': CD.loc[(CD.index == s) & (CD['Img'] == i), 'Ysmooth'].values, 
                        'InitialS': CD.loc[(CD.index == s) & (CD['Img'] == i), 'InitialS'].values,
                        'Xcentered': Xcont,
                        'Ycentered': Ycont,
                        'XAligned': Xcontal*Scale,
                        'YAligned': Ycontal*Scale} 
            
            newCD = newCD.append(pd.DataFrame(data=data,index = np.repeat(s,len(Xcont))) )
            
            # create aligned parametrisation
            npts = len(Xcontal)
            
            S = np.linspace(0,1,npts)
            
            locn1al = np.argmin(np.abs(np.sqrt(np.square(Xcontal-Xn2al)+np.square(Ycontal-Yn2al))))
            locn2al = np.argmin(np.abs(np.sqrt(np.square(Xcontal-Xn1al)+np.square(Ycontal-Yn1al))))
            
            Sn1al = S[locn1al]
            Sn2al = S[locn2al]
            
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1Aligned'] = locn1al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2Aligned'] = locn2al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch1Aligned'] = Sn1al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2Aligned'] = Sn2al
            
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'XattachAligned'] = XAal*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'YattachAligned'] = YAal*Scale

            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1Aligned'] = Xn1al*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1Aligned'] = Yn1al*Scale

            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2Aligned'] = Xn2al*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2Aligned'] = Yn2al*Scale
        
            if SavedPlots|DebugPlots:
                fig,[ax0,ax1] = plt.subplots(ncols=2,dpi=200,facecolor='black')
                fig.suptitle(s)
                ax0.plot(Xcont,Ycont,'c')
                ax0.plot(XA,YA,'wo')
                ax0.plot(Xn1,Yn1,'r*',ms=5)
                ax0.plot(Xn2,Yn2,'m*',ms=5)
                ax0.plot(0,0,'b*')
                ax0.plot([0, Xn1],[0, Yn1],'m--')
                ax0.plot([0, Xn2],[0, Yn2],'r--')
                ax0.plot([0, Xn2],[0, Yn2],'r--')
                #ax0.set_xlim(left=-250,right=250)
                #ax0.set_ylim(top=250,bottom=-250)
                ax0.set_aspect('equal', adjustable='box')

                ax1.plot([-250,250],[0, 0],'w--',lw = 0.5)
                ax1.plot(Xcontal,Ycontal,'c')
                ax1.plot(XAal,YAal,'wo')
                ax1.plot(Xn1al,Yn1al,'r*',ms=5)
                ax1.plot(Xn2al,Yn2al,'m*',ms=5)
                ax1.plot(0,0,'b*')
                ax1.plot([0, Xn1al],[0, Yn1al],'m--')
                ax1.plot([0, Xn2al],[0, Yn2al],'r--')
                #ax1.set_xlim(left=-250,right=250)
                #ax1.set_ylim(top=250,bottom=-250)
                ax1.set_aspect('equal', adjustable='box')

                fig.tight_layout()
                fig.savefig(P + '\\Figures\\Rotation\\'+ s + '\\' + str(i) +'.png') 
                if DebugPlots & (i == 0):
                    plt.show()
                else:
                    plt.close()
        
    return(newCD,GD)

### Defining curvilinear-like abscisse with landmarks

In [ ]:
def curvAbsci(CD,GD,StackList,P, **kwargs):

    if not os.path.exists(P + '\\Figures\\Parametrisation\\'):
        os.mkdir(P + '\\Figures\\Parametrisation\\') # create folder
    
    DebugPlots = False
    SavedPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        if key == 'saveplots':
            SavedPlots = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    newCD = pd.DataFrame(data=None,columns=['Img','Xparam','Yparam','Sparam']) 
    
    
    for s in StackList:
        
        print('Processing ' + s + '...',end=' ')
        
        if not os.path.exists(P + '\\Figures\\Parametrisation\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Parametrisation\\'+ s + '\\') # create folder
            
        n = int(1 + np.max(CD.loc[s, 'Img']))
        
        for i in range(n): # 
            
            # retrieve important points and contours
            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'XAligned'].values
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'YAligned'].values
            
            S = CD.loc[(CD.index == s) & (CD['Img'] == i), 'InitialS'].values

            
            Xat = GD.loc[(GD.index == s) & (GD['Img'] == i), 'XattachAligned'].values
            Yat = GD.loc[(GD.index == s) & (GD['Img'] == i), 'YattachAligned'].values

            Xn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1Aligned'].values
            Yn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1Aligned'].values

            Xn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2Aligned'].values
            Yn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2Aligned'].values
            
            locn1 = int(GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1Aligned'].values)
            locn2 = int(GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2Aligned'].values)
            Sn1 = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch1Aligned'].values
            Sn2 = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2Aligned'].values
            
            # Reparametrisation based on landmarks
            
            if Sn1<Sn2:
                Sntmp = Sn1[:]
                Sn1 = Sn2[:]
                Sn2 = Sntmp[:]
                
                loctmp = locn1
                locn1 = locn2
                locn2 = loctmp
            
            VectSa = S[0:locn2+1]
            VectSb = S[locn2+1:locn1+1]
            VectSc = S[locn1+1:]

            Sn1_ = 0.75
            Sn2_ = 0.25
            
            VectSa_ = VectSa*Sn2_/Sn2
            VectSb_ = Sn2_ + (VectSb-Sn2)*(Sn1_-Sn2_)/(Sn1-Sn2)
            VectSc_ = Sn1_ + (VectSc-Sn1)*(1-Sn1_)/(1-Sn1)
            
            S_ = np.concatenate((VectSa_,VectSb_,VectSc_))
            
            if SavedPlots|DebugPlots:
                ##### Plotting
                top = cm.get_cmap('Blues_r', 128)
                bot = cm.get_cmap('Reds', 128)

                newcolors = np.vstack((bot(np.linspace(0, 1, 128)),
                                       top(np.linspace(0, 0.5, 128)),
                                      np.flip(top(np.linspace(0, 0.5, 128)),axis=0),
                                       np.flip(bot(np.linspace(0, 1, 128)),axis=0)
                                      ))
                newcmp = ListedColormap(newcolors, name='RedBlue')

            
                fig0, [ax,ax2] = plt.subplots(ncols = 2,dpi = 200,facecolor = 'black')
                fig0.suptitle(s)
                ax.set_title('Original')
                sc0 = ax.scatter(Xcont,Ycont,c=S,cmap=newcmp,s=3)
                ax.plot(Xn1,Yn1,'c+',ms=4)
                ax.plot(Xn2,Yn2,'c+',ms=4)
                ax.set_aspect('equal', adjustable='box')
                ax2.set_title('Corrected')
                sc1 = ax2.scatter(Xcont,Ycont,c=S_,cmap=newcmp,s=3)
                ax2.plot(Xn1,Yn1,'c+',ms=4)
                ax2.plot(Xn2,Yn2,'c+',ms=4)
                ax2.set_aspect('equal', adjustable='box')
                fig0.tight_layout()
                cbar0 = fig0.colorbar(sc0, ax=[ax,ax2],ticks=[0, 0.25, 0.5, 0.75, 1],orientation='horizontal')
                cbar0.set_label('Landmark parametrisation')

                fig0.savefig(P + '\\Figures\\Parametrisation\\'+ s + '\\' + str(i) +'.png') 
                if DebugPlots & (i == 0) : # 
                    plt.show()
                else:
                    plt.close()

                fig,[ax0,ax1] = plt.subplots(nrows = 2,dpi = 200,facecolor = 'black')
                fig.suptitle(s)
                ax0.plot(S,Xcont,'r')
                ax0.plot(S_,Xcont,'b')
                ax0.set_ylabel('X coordinate')
                ax0.legend(['Original parametrisation','Corrected parametrisation'])
                ax1.plot(S,Ycont)
                ax1.plot(S_,Ycont,'b')
                ax1.set_ylabel('Y coordinate')
                ax1.set_xlabel('Landmark parametrisation')

                fig.tight_layout()
                fig.savefig(P + '\\Figures\\Parametrisation\\'+ s + '\\' + 'XY_' + str(i) +'.png') 
                if DebugPlots & (i == 0):
                    plt.show()
                else:
                    plt.close()

            
            
            ##########  cut different contour parts in fixed number of points separated by the same distance (per contour parts)
            
            ContourLength = np.sum(np.sqrt(np.square(np.diff(Xcont))+np.square(np.diff(Ycont))))
            ContourCumLength = np.concatenate(([0],np.cumsum(np.sqrt(np.square(np.diff(Xcont))+np.square(np.diff(Ycont))))))

            npts = 250 # number of point per section of contour
            
            # Attach point (0) to first notches clockwise (2)
            SegmentLength_02 = ContourCumLength[locn2]
            deltaS_02 = SegmentLength_02/npts
            SegmentRegCumLength_02 = np.linspace(0,npts,npts+1)*deltaS_02
            keptPtsLoc_02 = [np.argmin(np.abs(ContourCumLength-SegmentRegCumLength_02[i])) for i in range(npts)]

            Xcurv_02 = Xcont[keptPtsLoc_02]
            Ycurv_02 = Ycont[keptPtsLoc_02]
            Scurv_02 = S_[keptPtsLoc_02]
            
            CurvContourLength_02 = np.sum(np.sqrt(np.square(np.diff(Xcurv_02))+np.square(np.diff(Ycurv_02))))
            
            # First notche (2) to second notche (1) clockwise
            SegmentLength_21 = ContourCumLength[locn1]-ContourCumLength[locn2]
            deltaS_21 = SegmentLength_21/(npts*2) # x2 as there is 'half' of the gemma contour between the two notches
            SegmentRegCumLength_21 = np.linspace(0,npts*2,npts*2+1)*deltaS_21+SegmentLength_02
            keptPtsLoc_21 = [np.argmin(np.abs(ContourCumLength-SegmentRegCumLength_21[i])) for i in range(npts*2)]

            Xcurv_21 = Xcont[keptPtsLoc_21]
            Ycurv_21 = Ycont[keptPtsLoc_21]
            Scurv_21 = S_[keptPtsLoc_21]
            
            CurvContourLength_21 = np.sum(np.sqrt(np.square(np.diff(Xcurv_21))+np.square(np.diff(Ycurv_21))))
            
            
            # Second notche (1) to attach pont (0) clockwise
            SegmentLength_10 = ContourCumLength[len(ContourCumLength)-1]-ContourCumLength[locn1]
            deltaS_10 = SegmentLength_10/(npts)
            SegmentRegCumLength_10 = np.linspace(0,npts,npts+1)*deltaS_10+SegmentLength_21+SegmentLength_02
            keptPtsLoc_10 = [np.argmin(np.abs(ContourCumLength-SegmentRegCumLength_10[i])) for i in range(npts)]

            Xcurv_10 = Xcont[keptPtsLoc_10]
            Ycurv_10 = Ycont[keptPtsLoc_10]
            Scurv_10 = S_[keptPtsLoc_10]
            
            CurvContourLength_10 = np.sum(np.sqrt(np.square(np.diff(Xcurv_10))+np.square(np.diff(Ycurv_10))))
            
            if DebugPlots:
                print(s + '_' + str(i) + ' contour length variation : ' + 
                      str(round((ContourLength-(CurvContourLength_02+CurvContourLength_21+
                                                CurvContourLength_10+deltaS_02+deltaS_21+deltaS_10))
                                /ContourLength*10000)/100) + '%')

            Xcurv = np.concatenate((Xcurv_02, Xcurv_21,Xcurv_10))
            Ycurv = np.concatenate((Ycurv_02, Ycurv_21,Ycurv_10))
            Scurv = np.concatenate((Scurv_02, Scurv_21,Scurv_10))
            
            if SavedPlots|DebugPlots:
                fig1, ax1 = plt.subplots(dpi = 200,facecolor='black')
                fig1.suptitle(s)
                ax1.set_title("Regular parametrisation by segment (1/10 pts)")
                ax1.plot(Xcont,Ycont,'w-')
                ax1.plot(Xcurv_02[1:len(Xcurv_02)-1:10],Ycurv_02[1:len(Xcurv_02)-1:10],'ro',ms = 3)
                ax1.plot(Xcurv_21[1:len(Xcurv_21)-1:10],Ycurv_21[1:len(Xcurv_21)-1:10],'go',ms = 3)
                ax1.plot(Xcurv_10[1:len(Xcurv_10)-1:10],Ycurv_10[1:len(Xcurv_10)-1:10],'bo',ms = 3)
                ax1.set_aspect('equal', adjustable='box')
                fig1.savefig(P + '\\Figures\\Parametrisation\\'+ s + '\\' + 'Reg_' + str(i) +'.png') 
                if DebugPlots & (i == 0):
                    plt.show()
                else:
                    plt.close()
                
            # Contour normalisation
            Thp,Rp = vf.ToCirc(Xcurv,Ycurv)
            XcN,YcN = vf.ToCart(Thp,np.divide(Rp,np.median(Rp)))
            
            # Storing contour data
            data = {'Img': i*np.ones(len(Xcurv)),
                        'Xparam': Xcurv,
                        'Yparam': Ycurv,
                        'XparamNorm': XcN,
                        'YparamNorm': YcN,
                        'Sparam': Scurv} 
            
            newCD = newCD.append(pd.DataFrame(data=data,index = np.repeat(s,len(Xcurv))) )            
        print('Done')
            
    return(newCD)

### Wrapper function for parametric contour computation

In [ ]:
def ParametriseContour(stringName,Path,dateCond,StackList,Scale,Todo, **kwargs):
    
    doL   = False
    doLRP = False
    doRP  = False
    doP   = False
    
    if Todo == 'LRP':        
        doLRP = True
    elif Todo == 'RP':
        doRP  = True
    elif Todo == 'P':
        doP   = True
    elif Todo == 'L':
        doL   = True
        
    DebugPlots = False  
    LdmkPlots = True
    Dmax = 20    
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'ldmkplots':
            SavedPlots = value
        elif key == 'Dmax':
            Dmax = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    print('\033[1m' + '\033[4m' + '\nAnalyzing ' + dateCond + ':\n' + '\033[0m')
    
    if doLRP|doL:
        ### Loading area and contour data
        ContourData = pd.read_csv(Path + '\\ContourData' + stringName + '_AreaCont.csv', index_col = 'Ind')
        GlobalData = pd.read_csv(Path + '\\GlobalData' + stringName + '_AreaCont.csv', index_col = 'Ind')


        print('\n\n\nGetting landmarks for : ' + dateCond + '\n\n')
        ContourData_LM, GlobalData_LM = getLandmarks(ContourData,GlobalData,StackList,Path, 
                                                     debug = DebugPlots, saveplots = LdmkPlots, Dmax = Dmax)
        if doLRP:
            GlobalData_LM.to_csv(Path + '\\GlobalData' + stringName + '_Landmarks.csv',index_label = 'Ind')
            ContourData_LM.to_csv(Path + '\\ContourData' + stringName + '_Landmarks.csv',index_label = 'Ind')
            print('\nLandmarks saved.\n\n')

    elif doRP:
        ### Loading landmarks
        GlobalData_LM = pd.read_csv(Path + '\\GlobalData' + stringName + '_Landmarks.csv',index_col = 'Ind')
        ContourData_LM = pd.read_csv(Path + '\\ContourData' + stringName + '_Landmarks.csv',index_col = 'Ind')

    if doLRP|doRP:
        print('\n\n\nAligning contours for : ' + dateCond + '\n\n')
        ContourData_RC,GlobalData_RC = rotateAndCenterShape(ContourData_LM,GlobalData_LM,StackList,Path,Scale, debug = DebugPlots)
        GlobalData_RC.to_csv(Path + '\\GlobalData' + stringName + '_Aligned.csv',index_label = 'Ind')
        ContourData_RC.to_csv(Path + '\\ContourData' + stringName + '_Aligned.csv',index_label = 'Ind')
        print('\nNon parametric contours saved.\n\n')

    elif doP:
        ### loading non parametric contours
        GlobalData_RC = pd.read_csv(Path + '\\GlobalData' + stringName + '_Aligned.csv',index_col = 'Ind')
        ContourData_RC = pd.read_csv(Path + '\\ContourData' + stringName + '_Aligned.csv',index_col = 'Ind')

    if doLRP|doRP|doP:        
        print('\n\n\nComputing parametric contours for : ' + dateCond + '\n\n')
        ContourData_Param = curvAbsci(ContourData_RC,GlobalData_RC,StackList,Path, debug = DebugPlots)
        ContourData_Param.to_csv(Path + '\\ContourData' + stringName + '_Param.csv',index_label = 'Ind')
        print('\nParametric contours saved.\n\n')
        
    else:
        print('No analysis done for : ' + dateCond)
        
    return

##  Enter data and run

### Data details

In [ ]:
Scale210903 = 1.94 # Spatial scale (µm/px) for 16X 03-09-2021
FPH210903 = 2 # Frames per hour

Scale210914 = 1.24 # Spatial scale (µm/px) for 25X 14-09-2021
FPH210914 = 2 # Frames per hour

Scale210927 = 1.24 # Spatial scale (µm/px) for 25X 27-09-2021
FPH210927 = 2 # Frames per hour

Scale211022 = 0.97 # Spatial scale (µm/px) for 32X 27-09-2021
FPH211022 = 2 # Frames per hour

Scale211105 = 1.55 # Spatial scale (µm/px) for 20X 05-11-2021
FPH211105 = 2 # Frames per hour

Scale211222 = 1.55 # Spatial scale (µm/px) for 20X 22-12-2021
FPH211222 = 2 # Frames per hour

Scale220107 = 1.94 # Spatial scale (µm/px) for 16X 07-01-2022
FPH220107 = 2 # Frames per hour

Scale220112 = 1.94 # Spatial scale (µm/px) for 16X 07-01-2022
FPH220112 = 2 # Frames per hour

Scale220114 = 1.94 # Spatial scale (µm/px) for 16X 07-01-2022
FPH220114 = 2 # Frames per hour

# Paths to data
P210903_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\1mlh'
P210903_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\500ulh'
P210914_Ct = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Ctrl'
P210914_M125 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Manitol125'
P210927_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\1mlh'
P210927_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\500ulh'
P211022_Ct1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl1'
P211022_Ct2 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl2'
P211105_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\BSA'
P211105_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\NoCoat'
P211222_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\BSA'
P211222_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\NoCoat'
P220107_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade3'
P220107_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade1'
P220112_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade3'
P220112_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade1'
P220114_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade3'
P220114_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade1'


# experiments of 03-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210903_1 = ['PPG1','PPG2','PPG3','PPG6','PPG7','PPG10','PPG11','PPG12','PPG13','PPG14','PPG17','PPG19',
                   'PPG21','PPG22'] # 'PPG4','PPG5','PPG8','PPG9','PPG15','PPG16','PPG18','PPG20'

# experiments of 03-09-2021, Puce V5, Ctrl with 500ul/heure medium flow
StackList210903_5 = ['PPG3','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11'] # 'PPG1','PPG2','PPG4'

# experiments of 14-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210914_Ct = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10','PPG11'] # 'PPG8','PPG12'

# experiments of 14-09-2021, Puce V5, 125mM manitol with 1ml/heure medium flow
StackList210914_M125 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG7','PPG8','PPG9'] # 'PPG6'


# experiments of 27-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210927_1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG7','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15',] # 'PPG6','PPG8','PPG9'

# experiments of 27-09-2021, Puce V5, Ctrl with 500ul/heure medium flow
StackList210927_5 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG8','PPG9','PPG10','PPG11','PPG13'] #'PPG7','PPG12'


# experiments of 22-10-2021, double puce V6, Ctrl 1 with 500ul/heure medium flow
StackList211022_Ct1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19',
                       'PPG20','PPG21','PPG22','PPG23','PPG24','PPG25'] 
                    # 'PPG10', 
    
# experiments of 22-10-2021, double puce V6, Ctrl 2 with 500ul/heure medium flow
StackList211022_Ct2 = ['PPG1','PPG2','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG15','PPG16','PPG18','PPG19','PPG20',
                   'PPG22','PPG23','PPG24','PPG26','PPG27','PPG28','PPG29']
                    # 'PPG3',  'PPG14', 'PPG17', 'PPG21', 'PPG25',


# experiments of 05-11-2021, double puce V6, Degased medium, BSA coated channel with 500ul/heure medium flow
StackList211105_Deg_Bsa = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10',
                     'PPG11', 'PPG13','PPG16','PPG18','PPG19','PPG20',
                   'PPG21','PPG23','PPG24','PPG25','PPG26','PPG27','PPG28','PPG29','PPG30']  #  
                       # PPG12, PPG8 dead , PPG15, PPG17 dead notch, PPG22 dead parts, PPG14 folded lost attach

# experiments of 05-11-2021, double puce V6, Degased medium  with 500ul/heure medium flow
StackList211105_Degas = ['PPG1','PPG2','PPG3','PPG4','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG16','PPG17','PPG18','PPG19','PPG20'] # 
                # PPG3 dead attach, PPG5 dead notch, 'PPG15' dead attach

    
# experiments of 22-12-2021, double puce V6, degased medium, bsa coating with 500ul/heure medium flow
# /!\ /!\ Start 30 minutes later than usual, first image removed
StackList211222_Deg_Bsa = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG22','PPG23','PPG24','PPG25','PPG26','PPG28','PPG29','PPG30',
                          'PPG31'] # PPG22 ,'PPG13' touche le bord de l'image # ,'PPG27' bad contouring


# experiments of 22-12-2021, double puce V6, degased medium, no coating with 500ul/heure medium flow
# /!\ /!\ Start 30 minutes later than usual, first image removed
StackList211222_Degas = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG22','PPG23','PPG24','PPG25','PPG26','PPG27','PPG28','PPG29','PPG30']
                    # PPG8 trop plié


# experiments of 07-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220107_S1 = ['PPG1','PPG3','PPG5','PPG6','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG22','PPG23','PPG24','PPG25','PPG26','PPG27']
                    # ,'PPG2','PPG7' contact avec autre PPG

# experiments of 07-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220107_S3 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20']

# experiments of 12-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220112_S1 = ['PPG1','PPG2','PPG3','PPG5','PPG6','PPG8','PPG9','PPG10','PPG11','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50','PPG51','PPG52','PPG53','PPG54','PPG55','PPG56','PPG57','PPG58','PPG59','PPG60','PPG61',
                      'PPG62','PPG63','PPG64','PPG65','PPG66','PPG67','PPG68']
                    # PPG30 erreur de création de la stack #,'PPG7' ppg hors champ image 0
                    # ,'PPG4' same than 1,  'PPG12', same than 8

# experiments of 12-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220112_S3 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50']
                        

# experiments of 14-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220114_S1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50','PPG51']
                
# experiments of 14-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220114_S3 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG3233']
    
print('Data choice made.')


### Run by experiment

#### Flux controls V5 (03/27-09-2021)

#### Manitol 125mM V5 (14-09-2021)

#### System control DV6 (22-10-2021)

#### Degas & BSA controls DV6 (05-11-2021, 22-12-2021)

#### Stade comparison DV6 (07/12/14-01-2022)

In [ ]:
# 07-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

#ParametriseContour('220107_S1',P220107_S1,'07-01-2022 Stade 1 cups',StackList220107_S1,Scale220107,'L',ldmkplots=True)

#ParametriseContour('220107_S3',P220107_S3,'07-01-2022 Stade 3 cups',StackList220107_S3,Scale220107,'L',ldmkplots=True)

# 12-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

StackList220112_S1 = ['PPG60']

ParametriseContour('220112_S1',P220112_S1,'12-01-2022 Stade 1 cups',StackList220112_S1,Scale220112,'L',ldmkplots=True)

ParametriseContour('220112_S3',P220112_S3,'12-01-2022 Stade 3 cups',StackList220112_S3,Scale220112,'L',ldmkplots=True)

# 14-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

ParametriseContour('220114_S1',P220114_S1,'14-01-2022 Stade 1 cups',StackList220114_S1,Scale220114,'L',ldmkplots=True)

ParametriseContour('220114_S3',P220114_S3,'14-01-2022 Stade 3 cups',StackList220114_S3,Scale220114,'L',ldmkplots=True)

## Test Zone

In [ ]:
#%run D:/Users/laplaud/Desktop/PostDoc/Code/Marchantia/PropaguleAnalysis/3_ShapeQuantificationNB_Light.ipynb

In [ ]:
a = np.array([[1, 2, 3, 4, 5]])
b = np.transpose(np.array([[1,1,1,1,1,1,1]]))

In [ ]:
print(np.size(np.empty(360)))

In [ ]:

                    ThetaAOld,RAOld = vf.ToCirc(xAOld,yAOld, angle = 'deg')
                    xAOldRot,yAOldRot = vf.ToCart(ThetaAOld+m,RAOld, angle = 'deg')

In [ ]:
print(np.transpose(np.array([range(0,10)])))

In [ ]:
c = mtl.repmat(b,1,3)
print(c)